In [1]:

import os,json
import gc
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch import Tensor
import torch.utils.data as data
import torchvision
from torchvision import transforms
from torchvision import models
import torchvision.models as models
import matplotlib.pyplot as plt
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'

In [2]:

"""
error index + filter_input F4F에 넣어서 offset을 뽑아서 돌려보기

"""
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)
random.seed(0)

pretrained_model = models.vgg16_bn(pretrained=True)
#summary(pretrained_model, (3, 224, 224)) # (channels, width, height)
pretrained_model.cuda() # model에 GPU 할당
print(pretrained_model)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [3]:
!ls /media/1/Imagenet_dup

imagenet_class_index.json  train  val


In [4]:
new_model=models.vgg16_bn(pretrained=True).cuda()

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

transforms_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), # numpy를 tensor(torch)로 바꾸어 넣는다. 이미지의 경우 픽셀 값 하나는 0 ~ 255 값을 갖는다. 하지만 ToTensor()로 타입 변경시 0 ~ 1 사이의 값으로 바뀜.
    normalize,
])

transforms_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224), # 바로 resize 하지 말고 여기서 input format으로 맞춤.
    transforms.ToTensor(),
    normalize,
])

file = open('Filter_for_Filter_result.txt', 'w')    # hello.txt 파일을 쓰기 모드(w)로 열기. 파일 객체 반환
num_epochs = 20
batchsize = 64
lr = 0.001
class_num=1000 # class 개수
channel_per_packet=2 # channel당 packet 수 (pooling5에서 자르면 8이 된다.)
packet_loss_per_feature=32 # feature의 총 256개 packet 중에서 packet loss 개수 (즉, feature당 channel loss는 512개 중에서 2 * 64로 128개가 loss 된다.)

TRAIN_DATA_PATH = "/media/1/Imagenet_dup/train"
TEST_DATA_PATH="/media/1/Imagenet_dup/val"

### data loader ###

trainset = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=transforms_train) # numpy를 Tensor로 바꾸어 넣는다.
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batchsize, shuffle=True,num_workers=4)
testset = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=transforms_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batchsize, shuffle=True, num_workers=4)
# num_workers : how many subprocesses to use for data loading. 0 means that the data will be loaded in the main process. (default: 0)


In [5]:
loss_start_index=0
# hook 설정!
def init_weights(m): # 이거는 weight를 모두 0.5로 초기화 하는 부분 (여기서는 필요없는 부분)
    if type(m) == nn.Linear:
        m.weight.data.fill_(0.5)

def print_grad(name): # 이거는 backward 에서 쓰는 hook. (여기서는 필요없는 부분)
    def hook(model, input, output):
        print('BACKWARD', name, input, output)
    return hook

def preprocessing(name): # pre_hook 부분
    def hook(model, input): # pre hook은 해당 layer의 pre processing 부분이기에 output이 없어서 output을 parameter로 놓지 않는다.
        input[0][:,loss_start_index:loss_start_index+channel_per_packet*packet_loss_per_feature] = 0 # channel loss 발생! (input이 2D이기에 input[0]으로 해줘야 우리가 생각하는 feature이다.)
    return hook

def print_forward(name): # forward hook 부분 (여기에서는 필요 없음)
    def hook(model, input, output):
        if name=="module.features.34":
            return output
    return hook

class F4F_only_error_index(nn.Module): # error index만 넣어서 offset(3 x 3 x 512) 뽑는다. offset은 512개 각 filter에 동일하게 적용된다. 
    def __init__(self):
        super(F4F_only_error_index, self).__init__()
        self.fc1=nn.Linear(3*3*512+512,3*3*512) # input : error bit vector(512), output : 3 x 3 x 512 (filter offset)
    def forward(self,x): # x는 data를 나타낸다.
        x=self.fc1(x) # data가 fc를 지나간다.
        output=torch.tanh(x)
        return output

class SaveOutput:
    def __init__(self):
        self.outputs = []
        
    def __call__(self, module, module_in, module_out):
        self.outputs.append(module_out)
        if(len(self.outputs)>2): # 누적되면 앞의 것을 지운다.
            del self.outputs[:2]
        
    def clear(self):
        self.outputs = []
          

In [6]:
criterion = nn.MSELoss().cuda() # cost function 
pretrained_model.features._modules
conv5_1_layer = pretrained_model.features._modules['34']
print(conv5_1_layer.weight.size())

torch.Size([512, 512, 3, 3])


In [7]:

for start_index in range(0,3*128+1,128): 
    before_accuracy=0.0
    F4F=F4F_only_error_index() # model 선언
    F4F.cuda()
    optimizer = torch.optim.SGD(F4F.parameters(), lr=lr, weight_decay=1e-4) # optimizer
    loss_start_index=start_index # 0~127, 128~255, 256~383, 384~511
    error_index=[] # error 위치는 1, 아닌것은 0 (channel 위치)
    for index in range(512):
        if loss_start_index<=index and index < loss_start_index+channel_per_packet*packet_loss_per_feature: # error 위치
            error_index.append(0)
        else:
            error_index.append(1)
    error_index=torch.Tensor(error_index)
    error_index  = error_index.unsqueeze(0).repeat(512,1)
    error_index=error_index.cuda() # 512 bit vector (error location)
    flatten_weight = torch.reshape(conv5_1_layer.weight,(512,512*3*3))
    #print(error_index.size(),flatten_weight.size())
    info = torch.cat( (flatten_weight,error_index), 1 )
    #print(info.size())
    
    for epoch in range(num_epochs): # epoch 20번
        F4F.train()
        #optimizer.zero_grad() #  autograd에서 gradient가 축적 되기 때문에, gradient를 통해 가중치들을 업데이트할 때마다, 다음으로 넘어가기 전에 이 zero_() 메소드를 통해 gradient를 0로 만들어 줘야한다. 
        result = F4F(info) # result : 4608의 Tensor 형태 (512 x 3 x 3) => filter의 offset으로 사용될 예정. 더하고 빼는 경우 2가지 다 해보자.
        result=torch.reshape(result,[512,512,3,3])  # filter에 넣을 형태로 변경
        print(result.size())
        exit(0)
        #### filter를 변경한 새로운 모델 ####
        for name, parameter in new_model.named_parameters():
            if name == 'module.features.34.weight': # 바꿀 filter
                parameter=parameter[:,]+result # 새로운 filter
                new_model.module.features[34].weight.data = parameter # filter 변경!
                break       
        for name, module in new_model.named_modules():
            if name=="module.features.34": # conv 5-1 pre hook 걸기 (input feature에 error 넣기)
                module.register_forward_pre_hook(preprocessing(name)) # pre hook (forward)
                break

        save_output1 = SaveOutput()
        save_output2 = SaveOutput()
        
        original_output=[]
        error_output=[]
        #### 기존 모델에서 forward hook 통해서 conv5_1의 output 저장 ###
        for name, module in pretrained_model.named_modules():
            if name=="module.features.36": # conv 5-1 위치 (ReLU까지 통과시킨 것)
                original_output=module.register_forward_hook(save_output1) # forward hook 
                break
            
        #### 새로운 모델에서 conv5_1의 결과 뽑기 ####
        for name, module in new_model.named_modules():
            if name=="module.features.36": # 
                error_output=module.register_forward_hook(save_output2) # forward hook 
                break
        
        ####### train #######
        for idx, (images, labels) in enumerate(trainloader):
            images = images.cuda()
            out1=pretrained_model(images)
            out2=new_model(images)
            
            ##### 중간 feature 뽑기 #####
            original_label=save_output1.outputs
            original_label=torch.Tensor([item.cpu().detach().numpy() for item in original_label]).cuda() 
            """
                This is because gpu Upper tensor You can't go straight to numpy;  
                You need to be in  cpu  Complete the operation on , Back to  gpu  On
                If it's in  cpu  On , above .cpu()  and .cuda()  It can be omitted
            """
            original_label=torch.reshape(original_label,[batchsize,512,14,14]) # 이상하게 512, 14, 14는 제대로 나오는데 앞쪽이 쪼개져서 나와서 이렇게 모양 바꿈
            
            error_label=save_output2.outputs
            error_label=torch.Tensor([item.cpu().detach().numpy() for item in error_label]).cuda()
            error_label=torch.reshape(error_label,[batchsize,512,14,14]) # 이상하게 512, 14, 14는 제대로 나오는데 앞쪽이 쪼개져서 나와서 이렇게 모양 바꿈
            
            #### 학습 #####
            
            original_label=original_label.cuda()
            error_label=error_label.cuda()
            original_label.requires_grad_(True) # requires 
            error_label.requires_grad_(True)
            optimizer.zero_grad() #  autograd에서 gradient가 축적 되기 때문에, gradient를 통해 가중치들을 업데이트할 때마다, 다음으로 넘어가기 전에 이 zero_() 메소드를 통해 gradient를 0로 만들어 줘야한다. 
            loss = criterion(error_label,original_label) # MSE cost function 적용
            loss.backward() # autograd 를 사용하여 역전파 단계를 계산합니다. 이는 requires_grad=True를 갖는 모든 텐서들에 대한 손실의 변화도를 계산합니다.
            optimizer.step()
            if idx>400:
                break
        
        ###### test (새로운 filter가 들어간 vgg16) ######             
        print("Test start!!!")
        new_model.eval() # Change model to 'eval' mode (BN uses moving mean/var)

        correct_top1 = 0
        total = 0
        with torch.no_grad(): # 이 컨텍스트 내부에서 새로 생성된 텐서들은 requires_grad=False 상태가 되어, 메모리 사용량을 아껴준다. (훈련 안함)
            for idx, (images, labels) in enumerate(testloader):
                images = images.cuda()
                labels = labels.cuda()
                outputs = new_model(images)
                _, predicted = torch.max(outputs, 1) # top 1 기준
                total += labels.size(0) # labels.size : batch size가 64이니 64이다. (맨 끝에만 16개)
                correct_top1 += (predicted == labels).sum().item()
                print("step : {} / {}".format(idx + 1, len(testset)/int(labels.size(0))))
                print("top-1 percentage :  {0:0.2f}%".format(correct_top1 / total * 100))
        file.write("error channel {0}~{1}, epoch : [{2}/{3}]\n".format(loss_start_index,loss_start_index+channel_per_packet*packet_loss_per_feature-1, epoch+1, num_epochs))        
        file.write("top-1 percentage :  {0:0.2f}%\n".format(correct_top1 / total * 100))
        if (correct_top1 / total * 100) < before_accuracy:
            optimizer = torch.optim.SGD(F4F.parameters(), lr=before_lr*0.5, weight_decay=1e-4) # optimizer (아까 뽑아낸 훈련시킬 것만 설정)  => 안쓰는 것들은 자동으로 freeze
            before_lr=before_lr*0.5
        before_accuracy=(correct_top1 / total * 100)
        
        ###### 새로운 filter 저장 #######
        for name, parameter in new_model.named_parameters():
            if name == 'module.features.34.weight': # 훈련시킬 것만 뽑는다.
                num=str(loss_start_index).zfill(3)
                num2=str(loss_start_index+channel_per_packet*packet_loss_per_feature-1).zfill(3)
                epoch_num=str(epoch+1).zfill(2)
                torch.save(parameter, f"/media/3/Network/filter/pooling4/only_error_input_F4F/conv5_1_{num}~{num2}_train_epoch_{epoch_num}.pt") # 예를들어 128~255이면 128~255번째 channel이 깨진 것이다. 그리고 뒤의 숫자는 학습 횟수이다. 7이면 7번 epoch 돌린것
                break
    
                
file.close() # 파일 객체 닫기


torch.Size([512, 512, 3, 3])


RuntimeError: CUDA out of memory. Tried to allocate 98.00 MiB (GPU 0; 9.78 GiB total capacity; 8.40 GiB already allocated; 84.31 MiB free; 8.42 GiB reserved in total by PyTorch)

In [2]:
import torch.nn as nn
import torch
loss = nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)
output.backward()

In [3]:
print(output)

tensor(3.5468, grad_fn=<MseLossBackward>)


In [4]:
print(target)

tensor([[-0.0367,  0.6334,  0.7848,  0.1822,  0.4197],
        [ 1.8158,  0.8295, -2.4575,  1.3170,  0.5765],
        [ 0.3259, -0.5570,  2.1664, -0.9827, -0.6414]])


In [5]:
print(input)

tensor([[-0.3930,  0.7822, -0.7885,  1.0197, -0.6795],
        [-1.5242, -1.6168,  0.2296, -0.7026,  1.7953],
        [-0.8054,  1.2287, -0.5689,  1.4634,  0.2580]], requires_grad=True)
